In [144]:
import sys
import uproot

import os
import numpy as np
import pandas as pd

import argparse

In [145]:
part =   "electron" #"pion_c" #"gamma" #"muon"
path = "/data/user/adipilat/ParticleID/genEvts/"
dir_ = "ana"
tree = "hgc"

In [146]:
variableName = [
            'event','run','lumi',
            'cluster2d_layer',
            'cluster2d_energy',
            'cluster2d_eta',
            'cluster2d_phi',
            'cluster2d_pt',
            'cluster2d_x',
            'cluster2d_y',
            'cluster2d_z',
            'cluster2d_nhitCore',
            'cluster2d_nhitAll',
            'gen_energy',
            'gen_pdgid',
            'gen_daughters',
            'gen_phi',
            'gen_eta',
            'trackster_clusters'
            ]
newVars =["run","lumi","event","trackster","x","y","z","r","layer","E","nCore","nHits","id","genDR","gen_phi","gen_eta","phi","eta","pid"]

In [147]:
print("Starting data production")
name = "4_" + part
file = path + part + "/" + name + ".root"

Starting data production


In [148]:
df = uproot.open(file)[dir_][tree].pandas.df(variableName,flatten=False)


num_events = np.unique(df["event"].values).shape[0]
xs = df["cluster2d_x"].values
ys = df["cluster2d_y"].values
zs = df["cluster2d_z"].values
es = df["cluster2d_energy"].values
ps = df["cluster2d_pt"].values
nh = df["cluster2d_nhitAll"].values
nc = df["cluster2d_nhitCore"].values
ll = df["cluster2d_layer"].values
ee = df["event"].values
lu = df["lumi"].values
ru = df["run"].values
    
sizes = [x.shape[0] for x in xs]
indices = [np.full((a[0]),a[1]) for a in zip(sizes,range(len(sizes)))]
        #gendau = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_daughters"].values)]

cphi = df["cluster2d_phi"].values
ceta = df["cluster2d_eta"].values
gpid = df["gen_pdgid"].values

gphi = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_phi"].values)]
geta = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_eta"].values)]
gpid = [np.full((a[0]),a[1]) for a in zip(sizes,df["gen_pdgid"].values)]

tr = df["trackster_clusters"].values

In [149]:
idtrlist = []

for i in range(len(sizes)):
    idtrlist.append(np.array([0]*sizes[i]))
for i in range(len(tr)):
    for j in range(len(tr[i])):
        for item in tr[i][j]:
            idtrlist[i][item] = j + 1

idtr = np.array(idtrlist)

In [150]:
rs = [np.sqrt(f[0]**2+f[1]**2) for f in zip(xs,ys)]

drs = [np.sqrt((a[0]-a[1])**2 + (a[2]-a[3])**2) for a in zip(gphi,cphi,geta,ceta)]


In [151]:
XS = np.array([item for sublist in xs for item in sublist])
YS = np.array([item for sublist in ys for item in sublist])
ZS = np.array([item for sublist in zs for item in sublist])
RS = np.array([item for sublist in rs for item in sublist])
LL = np.array([item for sublist in ll for item in sublist])
ES = np.array([item for sublist in es for item in sublist])
NC = np.array([item for sublist in nc for item in sublist])
NH = np.array([item for sublist in nh for item in sublist])
II = np.array([item for sublist in indices for item in sublist])
DRS = np.array([item for sublist in drs for item in sublist])
GPHI = np.array([item for sublist in gphi for item in sublist])
GETA = np.array([item for sublist in geta for item in sublist])
GPID = np.array([item for sublist in gpid for item in sublist])
CPHI = np.array([item for sublist in cphi for item in sublist])
CETA = np.array([item for sublist in ceta for item in sublist])


SS = [np.full((s,),s) for s in sizes]
EE = [np.full((s,),i) for i,s in zip(ee,sizes)]
LU = [np.full((s,),i) for i,s in zip(lu,sizes)]
RU = [np.full((s,),i) for i,s in zip(ru,sizes)]


SS = np.array([item for sublist in SS for item in sublist])
EE = np.array([item for sublist in EE for item in sublist])
LU = np.array([item for sublist in LU for item in sublist])
RU = np.array([item for sublist in RU for item in sublist])

TR = np.array([item for sublist in idtr for item in sublist])

datas = np.vstack((RU,LU,EE,TR,XS,YS,ZS,RS,LL,ES,NC,NH,II,DRS,GPHI,GETA,CPHI,CETA,GPID)).T

In [152]:
df = pd.DataFrame(datas,columns=newVars)
df = df.sort_values(["event","trackster","layer","E"],ascending=[True,True,True,False]).reset_index(drop=True)

In [153]:
# max_perlayer = 10
# theIndex = list(df.groupby(["event","trackster","layer", "id"]).indices.values())
# theIndex = np.array([item for sublist in theIndex for item in sublist[:min(len(sublist),max_perlayer)]])
# df = df.iloc[theIndex]
# df.reset_index(inplace=True, drop=True)

In [154]:
df.to_hdf(path + part + ".h5","data",complevel=0)

In [155]:
df.head(100)

,run,lumi,event,trackster,x,y,z,r,layer,E,nCore,nHits,id,genDR,gen_phi,gen_eta,phi,eta,pid
0,1.0,1.0,1.0,0.0,-56.716282,-33.819294,325.072754,66.033943,3.0,0.059753,2.0,2.0,0.0,0.011785,-2.601872,2.28558,-2.603906,2.297187,11.0
1,1.0,1.0,1.0,0.0,-57.043827,-34.008404,326.017273,66.412125,4.0,0.516198,1.0,1.0,0.0,0.009104,-2.601872,2.28558,-2.603987,2.294434,11.0
2,1.0,1.0,1.0,0.0,-57.043831,-34.149403,328.042755,66.484436,5.0,1.328837,7.0,7.0,0.0,0.013861,-2.601872,2.28558,-2.602165,2.299437,11.0
3,1.0,1.0,1.0,0.0,-57.791519,-36.109001,328.042755,68.144844,5.0,0.071703,1.0,1.0,0.0,0.021392,-2.601872,2.28558,-2.583127,2.275273,11.0
4,1.0,1.0,1.0,0.0,-62.027515,-34.411201,328.042755,70.933372,5.0,0.027221,1.0,1.0,0.0,0.059648,-2.601872,2.28558,-2.635092,2.236039,11.0
5,1.0,1.0,1.0,0.0,-57.043827,-34.143097,328.987244,66.481194,6.0,3.006564,9.0,9.0,0.0,0.016728,-2.601872,2.28558,-2.602246,2.302303,11.0
6,1.0,1.0,1.0,0.0,-60.582176,-44.165005,328.987244,74.971649,6.0,0.046474,1.0,1.0,0.0,0.135262,-2.601872,2.28558,-2.511661,2.184793,11.0
7,1.0,1.0,1.0,0.0,-62.725185,-24.341204,328.987244,67.282562,6.0,0.037180,1.0,1.0,0.0,0.169622,-2.601872,2.28558,-2.771421,2.290561,11.0
8,1.0,1.0,1.0,0.0,-57.923626,-34.732212,331.957275,67.538681,8.0,7.681228,20.0,20.0,0.0,0.010074,-2.601872,2.28558,-2.601452,2.295645,11.0
9,1.0,1.0,1.0,0.0,-54.253147,-36.425201,331.957275,65.346764,8.0,0.061081,1.0,1.0,0.0,0.066755,-2.601872,2.28558,-2.550325,2.327996,11.0
